In [9]:
import requests
import json
from string import Template

# LLM Setup

In [7]:
def ollama_generate(input: str, model = "zephyr-beta"):
    url = "http://localhost:11434/api/generate"
    
    payload = json.dumps({
      "model": model,
      "prompt": input,
      "stream": False
    })
    headers = {
      'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    return json.loads(response.text)['response'].strip()

In [11]:
chat_template = Template("""<|system|>
You are a friendly chatbot who always helpful, harmless and honest.</s>
<|user|>
$input</s>
<|assistant|>""")

prompt = chat_template.substitute(input="Hello!")
print(ollama_generate(prompt))

Hi there! I'm here to help you out with anything you need. Just let me know what you're looking for and I'll do my best to assist you. Have a great day!


# Tools

In [72]:
def web_search(input: str):
    # Use Serper API here
    return "Age of Tom Cruise is 42"

In [31]:
def meaning_of_life():
    return "42 is the answer to life, the universe, and everything."

In [52]:
def calculator():
    return "5"

# System State

In [64]:
system_state = {
    "task": "Calculate the double of the age of Tom Cruise",
    "tools": {
        "web_search": {
            "func": web_search,
            "description": "Useful to get information on an unknown topic by searching it online. Input must be a string.",
            # "return_direct": True
        },
        "meaning_of_life": {
            "func": meaning_of_life,
            "description": "Useful to get the meaning of life. No inputs needed for this tool.",
        },
        "calculator": {
            "func": calculator,
            "description": "Useful to perform basic arithmetic operations. Input must be a string.",
        },
    },
    "current_plan": [],
    "short_term_memory": [],
}

# Utility Functions

In [71]:
def get_current_tools():
    tool_str = ""
    for idx, tool in system_state['tools'].items():
        # print(idx, tool['description'])
        tool_str += f"Name: {idx}\nDescription: {tool['description']}\n\n"
    return tool_str.strip()

# print(get_current_tools())

# Re-planner Agent

**Basic idea**: Based on Plan-and-execute type agents, but capable of more complex tasks as it is able to think and update the plan as and when needed. This helps it to tackle tasks with longer steps.

Flow of the program,
1. `Think`: Keeping the current step in mind, choose the right tool from the arsenel with right input.
2. `Act`: Execute the selected tool with the input from the previous step.
3. `Observe`: Based on the output of the action step, extract relevant information.
4. `Update`: Based on the available data till now, re-think and update the plan if needed.

## Think

In [70]:
thought_template = Template("""<|system|>
You are an assistant who is always helpful, harmless and honest.</s>
<|user|>
Your task is to analyze complex tasks and break them down into smaller sub-tasks.

Instructions:
* Do not say your knowledge is out of date, just return the requested information.
* Do not say you are a AI language model.
* Do not perform the task just yet, analyze and break down the task and return the sub-tasks.
* If a tool is used, sub-task must say something like, 'Use tool_a with X to do Y'.
* Use only the tools mentioned below if and when needed.
* The sub-tasks must not say 'store the data'. All the data is by default stored in memory.
* Return the output in the form of an array, follow the below output format strictly.

Tools:
```
$tools
```

Output format:
```
[
    'sub-task 1', 
    'sub-task 2', 
    ...
]
```

Task: $input

Return the output in the specified format, do not deviate. Do not add any text before or after the output.</s>
<|assistant|>""")

prompt = thought_template.substitute(input="Calculate the double of the age of Tom Cruise", tools=get_current_tools())
# print(prompt)
print(ollama_generate(prompt))

[
  'Retrieve the current age of Tom Cruise using web_search',
  'Convert the retrieved age to a number using calculator with input "current age of tom cruise"',
  'Multiply the converted age by 2 using calculator with input "converted age * 2"'
]
